## Haar Cascade amb SVC (SVM)

In [ ]:
import itertools
import kagglehub
import os
import cv2
import numpy as np
import glob
import shutil
import pandas as pd
import dlib
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from itertools import zip_longest
from glob import glob
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from skimage.feature import hog
from skimage import exposure
import cv2
import os
import numpy as np
import mediapipe as mp
import pywt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm
import kagglehub
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.utils import to_categorical
import warnings; warnings.simplefilter('ignore')

Carreguem dataset

In [ ]:
source_dir = kagglehub.dataset_download("tapakah68/facial-emotion-recognition")
people_dirs = glob(os.path.join(source_dir+'/images', "*"))

# Path al dataset organitzat
dest_dir = "/kaggle/working/organized_by_emotion"
os.makedirs(dest_dir, exist_ok=True)

# Possibles emocions
emotions = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprised', 'Neutral', 'Contempt']

# Organitzem emocions per carpetes
for emotion in emotions:
    os.makedirs(os.path.join(dest_dir, emotion), exist_ok=True)


# Movem les emocions a les carpetes
for person_path in people_dirs:
    if os.path.isdir(person_path):
        person_id = os.path.basename(person_path)
        emotion_images = glob(os.path.join(person_path, "*.jpg"))

        for img_path in emotion_images:
            emotion = os.path.splitext(os.path.basename(img_path))[0].capitalize()
            if emotion in emotions:
                new_filename = f"{person_id}.jpg"
                dest_path = os.path.join(dest_dir, emotion, new_filename)
                shutil.copy(img_path, dest_path)

Busquem possibles boques i seleccionem la millor

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
mouth_cascade = cv2.CascadeClassifier('\Predictors\haarcascade_mcs_mouth.xml')
def extract_best_mouth(face_gray, mouths):
    """Boca centrada i cap avall de la cara."""
    h, w = face_gray.shape
    center_x, bottom_y = w // 2, int(h * 0.75)

    best_mouth = None
    min_dist = float('inf')
    for (mx, my, mw, mh) in mouths:
        mouth_cx = mx + mw // 2
        mouth_cy = my + mh // 2
        dist = (mouth_cx - center_x) ** 2 + (mouth_cy - bottom_y) ** 2
        if dist < min_dist:
            min_dist = dist
            best_mouth = (mx, my, mw, mh)
    return best_mouth

def extract_mouth_hog(face_gray):
    mouths = mouth_cascade.detectMultiScale(face_gray, 1.1, 4)
    if len(mouths) == 0:
        return None

    best_mouth = extract_best_mouth(face_gray, mouths)
    if best_mouth is None:
        return None

    mx, my, mw, mh = best_mouth
    mouth_roi = face_gray[my:my+mh, mx:mx+mw]
    resized = cv2.resize(mouth_roi, (64, 64))  # bigger for better HOG
    features = hog(resized, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)
    return features

Comencem entrenament amb SVC

In [ ]:
X_train, y_train = [], []
emotions = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprised', 'Neutral', 'Contempt']
DATA_DIR = '/kaggle/working/organized_by_emotion'
OUTPUT_FILE = 'mouth_vectors_all_emotions.pkl'


for emotion in emotions:
    emotion_path = os.path.join(DATA_DIR, emotion)
    files = sorted(os.listdir(emotion_path))[:12]  # 12 persones per train

    for filename in files:
        img_path = os.path.join(emotion_path, filename)
        img = cv2.imread(img_path)
        if img is None:
            continue
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        for (x, y, w, h) in faces:
            face_gray = gray[y:y+h, x:x+w]
            hog_features = extract_mouth_hog(face_gray)
            if hog_features is not None:
                X_train.append(hog_features)
                y_train.append(emotion)


le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)

clf = SVC(kernel='linear', probability=True)
clf.fit(X_train, y_train_enc)

with open('emotion_svm.pkl', 'wb') as f:
    pickle.dump((clf, le), f)

print(f"Train on {len(X_train)} samples.")

Test

In [ ]:
emotions = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprised', 'Neutral', 'Contempt']
DATA_DIR = '/kaggle/working/organized_by_emotion'

with open('emotion_svm.pkl', 'rb') as f:
    clf, le = pickle.load(f)

y_true, y_pred = [], []

for emotion in emotions:
    emotion_path = os.path.join(DATA_DIR, emotion)
    files = sorted(os.listdir(emotion_path))[12:18]  # ultimes 6 persones per test

    for filename in files:
        img_path = os.path.join(emotion_path, filename)
        img = cv2.imread(img_path)
        if img is None:
            continue
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        for (x, y, w, h) in faces:
            face_gray = gray[y:y+h, x:x+w]
            hog_features = extract_mouth_hog(face_gray)
            if hog_features is not None:
                pred = clf.predict([hog_features])[0]
                pred_label = le.inverse_transform([pred])[0]
                y_true.append(emotion)
                y_pred.append(pred_label)

# resultats
print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, zero_division=0))

cm = confusion_matrix(y_true, y_pred, labels=emotions)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=emotions, yticklabels=emotions)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix: SVM Mouth-Based Emotion Recognition')
plt.tight_layout()
plt.show()

## Haar Cascade amb distancia euclidieana

In [ ]:
# Set paths
DATA_DIR = '/kaggle/working/organized_by_emotion'
OUTPUT_FILE = 'mouth_vectors_all_emotions.pkl'

# List of emotions (must match folder names)
emotions = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprised', 'Neutral', 'Contempt']

emotion_vectors = []  # List of tuples: (vector, emotion_label)

def detect_mouths(gray_face):
    mouths = mouth_cascade.detectMultiScale(gray_face, 1.1, 4)
    mouth_vectors = []

    for (mx, my, mw, mh) in mouths:
        mouth_region = gray_face[my:my+mh, mx:mx+mw]
        resized = cv2.resize(mouth_region, (16, 16))
        vector = resized.flatten().astype(np.float32)
        mouth_vectors.append(vector)

    return mouth_vectors

for emotion in emotions:
    emotion_dir = os.path.join(DATA_DIR, emotion)
    if not os.path.exists(emotion_dir):
        print(f"Emocio {emotion} no existent")
        continue

    files = sorted(os.listdir(emotion_dir))[:12]

    for filename in files:
        file_path = os.path.join(emotion_dir, filename)

        img = cv2.imread(file_path)
        if img is None:
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        for (x, y, w, h) in faces:
            face_gray = gray[y:y+h, x:x+w]
            mouths = detect_mouths(face_gray)

            for vec in mouths:
                emotion_vectors.append((vec, emotion))

print(f"Guardats {len(emotion_vectors)} vectors.")

with open(OUTPUT_FILE, 'wb') as f:
    pickle.dump(emotion_vectors, f)

print(f"Dataset saved to {OUTPUT_FILE}")


In [ ]:
with open('mouth_vectors_all_emotions.pkl', 'rb') as f:
    train_vectors = pickle.load(f)

emotions = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprised', 'Neutral', 'Contempt']
DATA_DIR = '/kaggle/working/organized_by_emotion'

def detect_mouths(gray_face):
    mouths = mouth_cascade.detectMultiScale(gray_face, 1.1, 4)
    mouth_vectors = []
    for (mx, my, mw, mh) in mouths:
        mouth_region = gray_face[my:my+mh, mx:mx+mw]
        resized = cv2.resize(mouth_region, (16, 16))
        vector = resized.flatten().astype(np.float32)
        mouth_vectors.append(vector)
    return mouth_vectors

def find_closest_emotion(test_vec, train_vectors):
    min_dist = float('inf')
    predicted_emotion = 'unknown'
    for vec, label in train_vectors:
        dist = np.linalg.norm(test_vec - vec)
        if dist < min_dist:
            min_dist = dist
            predicted_emotion = label
    return predicted_emotion

y_true = []
y_pred = []

for emotion in emotions:
    emotion_dir = os.path.join(DATA_DIR, emotion)
    files = sorted(os.listdir(emotion_dir))[12:18]  
    for filename in files:
        file_path = os.path.join(emotion_dir, filename)
        img = cv2.imread(file_path)
        if img is None:
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        for (x, y, w, h) in faces:
            face_gray = gray[y:y+h, x:x+w]
            mouth_vecs = detect_mouths(face_gray)

            for mv in mouth_vecs:
                predicted = find_closest_emotion(mv, train_vectors)
                y_true.append(emotion)
                y_pred.append(predicted)

print(f"\n=== Classification Report ===\n")
print(classification_report(y_true, y_pred, zero_division=0))

cm = confusion_matrix(y_true, y_pred, labels=emotions)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=emotions, yticklabels=emotions)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Emotion Classification Confusion Matrix')
plt.show()